# 1. Setup

## 1.1 Install/Import Dependencies

In [ ]:
!pip list -v

In [ ]:
#Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
from picamera2 import Picamera2

In [ ]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

## 1.2 Create Folder Structure

In [ ]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [ ]:
# Make the directories
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)



## 2. Collect Positives and Anchors

In [ ]:
#!unzip '/home/nasir/FacialRecognitionSiameseModel/lfw.zip' -d '/home/nasir/FacialRecognitionSiameseModel/lfw'

In [ ]:
#Move LFW Images to the following Repository data/negative
'''for directory in os.listdir('/home/nasir/FacialRecognitionSiameseModel/lfw/lfw-deepfunneled/lfw-deepfunneled'):
    for file in os.listdir(os.path.join('/home/nasir/FacialRecognitionSiameseModel/lfw/lfw-deepfunneled/lfw-deepfunneled',directory)):
        EX_PATH = os.path.join('/home/nasir/FacialRecognitionSiameseModel/lfw/lfw-deepfunneled/lfw-deepfunneled',directory,file)
        NEW_PATH = os.path.join(NEG_PATH,file)
        os.replace(EX_PATH, NEW_PATH)'''

In [ ]:
# Import uuid library to generate unique image names
import uuid

In [ ]:
uuid.uuid1()

In [ ]:

# Initialize the camera
picam2 = Picamera2()
    
# Create a preview configuration (640x480 is good for testing)
config = picam2.create_preview_configuration(main={"format": "XRGB8888", "size": (250, 250)})
picam2.configure(config)
    
# Start the camera
picam2.start()
    
print("Press 'q' to quit the camera feed.")
    
# Continuously capture frames and display them
while True:
    frame = picam2.capture_array()
    #Collect anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
        
    #collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    cv2.imshow("PiCamera2 Feed", frame)
    
    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
# Clean up
cv2.destroyAllWindows()
picam2.stop()





In [ ]:
frame.shape